# Link features

So far, we haven't enabled ourselves to query for the features[^features] of ingested data, and used LaminDB like a data lake.

[^features]: We'll mostly use the term feature for synonyms variable (statistics), column and field (databases), dimension (machine learning).


We can also use LaminDB like a queryable data warehouse to store links[^relations] and monitor data integrity.

Let us explain how to implement this by providing feature models at ingestion!

[^relations]: We mostly use the term link for synonyms relations and references.

In [1]:
import lamindb as db
import bionty as bt  # https://lamin.ai/docs/bionty
import scanpy as sc  # https://scanpy.readthedocs.io

db.header()

id,ZKJX7AnXzQQp
version,7
time_init,2022-07-30 16:14
time_run,2022-08-28 21:15
consecutive_cells,True
pypackage,bionty==0.0.6+17.g0499eed lamindb==0.3.2 scanpy==1.8.2


## Example datasets

Consider
- `data1`: a flow cytometry dataset in form of an `.fcs` file
- `data2`: a scRNA-seq count matrix in form of an `AnnData` object in memory

In [2]:
data1 = db.datasets.file_fcs()
data1

PosixPath('example.fcs')

In [3]:
data2 = sc.read(db.datasets.file_mouse_sc_lymph_node())

## Define feature models

For `data1`, we specify a feature model using the `bionty` `Gene` entity with gene symbols.

In [4]:
feature_model1 = bt.CellMarker(species=bt.lookup.species.human)

Let us now ingest the data by passing a feature model to `db.do.ingest.add`, which will enable us to query the `dobject` by features by creating all necessary links in the background.

It will also log out and store information on data integrity:

In [5]:
db.do.ingest.add(data1, feature_model=feature_model1, featureset_name="flow_panel_1")

🔶 name column not found, using index as features.
✅ 13 terms (81.2%) are linked.
🔶 3 terms (18.8%) are not linked.


/opt/miniconda3/envs/py39/lib/python3.9/site-packages/fcsparser/api.py:440: UserWarning: The default channel names (defined by the $PnS parameter in the FCS file) were not unique. To avoid problems in downstream analysis, the channel names have been switched to the alternate channel names defined in the FCS file. To avoid seeing this warning message, explicitly instruct the FCS parser to use the alternate channel names by specifying the channel_naming parameter.
  warnings.warn(msg)
/opt/miniconda3/envs/py39/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Using this feature model, we can't link and hence won't be able to query for 9 features.

We can overcome this by working with a custom feature model, discussed later.

Features in data2 are indexed by Ensemble gene ids. For an overview of gene ids, see: [`bt.lookup.gene_id`](https://lamin.ai/docs/bionty/api).

In [6]:
data2.var.head()

""
ENSMUSG00000104923
ENSMUSG00000079038
ENSMUSG00000069755
ENSMUSG00000040648
ENSMUSG00000113486


Hence, we use a feature model based on Ensemble IDs and ingest the data with it.

In [7]:
feature_model2 = bt.Gene(
    id=bt.lookup.gene_id.ensembl_gene_id, species=bt.lookup.species.mouse
)

In [8]:
db.do.ingest.add(
    data2,
    name="mouse_sc_lymph_node",
    feature_model=feature_model2,
    featureset_name="mouse_1k",
)

🔶 ensembl_gene_id column not found, using index as features.
✅ 10000 terms (100.0%) are linked.
🔶 0 terms (0.0%) are not linked.


We can retrieve the integrity information through `.logs`:

In [9]:
db.do.ingest.logs

{'example.fcs': {'feature': 'name',
  'n_mapped': 13,
  'percent_mapped': 81.2,
  'unmapped': Index(['FSC-A', 'FSC-H', 'SSC-A'], dtype='object')},
 'mouse_sc_lymph_node.h5ad': {'feature': 'ensembl_gene_id',
  'n_mapped': 10000,
  'percent_mapped': 100.0,
  'unmapped': Index([], dtype='object')}}

Finalize the ingestion.

In [10]:
db.do.ingest.commit()

ℹ️ Added notebook 'Link features' (ZKJX7AnXzQQp, 7) by user test-user1.
✅ Registered readout: 1
✅ Annotated data d6aKmPAcmNsq1k45Vs0WZ with the following features:
+------------+------------+------------+
| geneset.id | biometa.id | species.id |
+------------+------------+------------+
|     1      |     1      |     1      |
+------------+------------+------------+


/opt/miniconda3/envs/py39/lib/python3.9/site-packages/sqlmodel/orm/session.py:60: SAWarning: Class SelectOfScalar will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(


✅ Registered readout: 2
✅ Annotated data sxE2YetgrHwrPQ2c35Pbz with the following features:
+------------+------------+------------+
| geneset.id | biometa.id | species.id |
+------------+------------+------------+
|     2      |     2      |     2      |
+------------+------------+------------+
✅ Ingested the following dobjects:
+-----------------------------------------------------+-----------------------------------+-----------------------+
|                       dobject                       |              jupynb               |         user          |
+-----------------------------------------------------+-----------------------------------+-----------------------+
|       example.fcs (d6aKmPAcmNsq1k45Vs0WZ, 1)        | 'Link features' (ZKJX7AnXzQQp, 7) | test-user1 (9ypQ1yrW) |
| mouse_sc_lymph_node.h5ad (sxE2YetgrHwrPQ2c35Pbz, 1) | 'Link features' (ZKJX7AnXzQQp, 7) | test-user1 (9ypQ1yrW) |
+-----------------------------------------------------+---------------------------------

🔶 Cells [(1, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None)] were not run consecutively.
